In [ ]:
#ในกรณีที่ยังไม่ได้แตกไฟล์
!unzip oh_my_law_database.zip

In [1]:
!pip install --upgrade transformers tokenizers huggingface_hub
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q bitsandbytes accelerate
!pip install langchain-community langchain-huggingface chromadb sentence-transformers
!pip install pdfplumber python-docx
!pip install pdfplumber python-docx


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
  Using cached transformers-5.0.0-py3-none-any.whl.metadata (37 kB)
  Using cached transformers-4.57.6-py3-none-any.whl.metadata (43 kB)
Using cached huggingface_hub-0.36.0-py3-none-any.whl (566 kB)
Using cached transformers-4.57.6-py3-none-any.whl (12.0 MB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 1.3.5
    Uninstalling huggingface_hub-1.3.5:
      Successfully uninstalled huggingface_hub-1.3.5
  Attempting uninstall: transformers
    Found existing installation: transformers 5.0.1.dev0
    Uninstalling transformers-5.0.1.dev0:
      Successfully uninstalled transformers-5.0.1.dev0


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
import pdfplumber
import unicodedata
from docx import Document
import os
import re

In [3]:
def load_modle():
  model_id = "Qwen/Qwen2.5-7B-Instruct"
  bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
    )
  print("กำลังโหลดและติดตั้ง Qwen2.5-7B")
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)
  print("download สำเร็จ")
  return [tokenizer,model]
load_modle()

กำลังโหลดและติดตั้ง Qwen2.5-7B


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

download สำเร็จ


[Qwen2TokenizerFast(name_or_path='Qwen/Qwen2.5-7B-Instruct', vocab_size=151643, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
 	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, single_word=Fal

In [4]:
def load_data():
  print("กำลังโหลดตัวแปลภาษา")
  embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-small")

  print("กำลังดึงข้อมูลจาก database")
  db = Chroma(persist_directory="./oh_my_law_database", embedding_function=embeddings)

  print("โหลดdata base สำเร็จ")

  return db
load_data()

กำลังโหลดตัวแปลภาษา


/tmp/ipython-input-749230657.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-small")


กำลังดึงข้อมูลจาก database


/tmp/ipython-input-749230657.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  db = Chroma(persist_directory="./oh_my_law_database", embedding_function=embeddings)


โหลดdata base สำเร็จ


In [5]:
def import_file(file_path):

    if not os.path.exists(file_path):
        print("Not found file")
        return

    ext = os.path.splitext(file_path)[1].lower()
    try:
        contract = ""

        if ext == ".pdf":
            contract = read_pdf(file_path)
            return contract
        elif ext == ".docx":
            contract = read_docx(file_path)
            return contract
        elif ext == ".txt":
            contract = read_txt(file_path)
            return contract
        else:
            print("worng type file")

    except Exception as e:
        print(f"error at read a file: {e}")

In [6]:
def read_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    return content

def read_docx(file_path):
    doc = Document(file_path)
    contract = []
    for para in doc.paragraphs:
        contract.append(para.text)
    return "\n".join(contract)

def read_pdf(file_path):
    contract = []
    with pdfplumber.open(file_path) as pdf:
        for i,page in enumerate(pdf.pages,start=1):
            chars = page.chars
            if not chars:
                print(f"หน้า {i} ไม่มีตัวอักษร")
                continue
            line = clean_pdf(chars)
            text = "".join(line)
            text = unicodedata.normalize("NFC", text)
            contract.append(text)
        return "\n".join(contract)

def clean_pdf(chars):
    THAI_MARKS = set(chr(c) for c in range(0x0E31, 0x0E4F))
    line = []

    for ch in chars:
        char = ch["text"]
        if char in THAI_MARKS and line:
            line[-1] += char
        else:
            line.append(char)
    return line

In [7]:
def read_contract(content):

    section_pattern = r'(\d+)\.\s+'
    footer_pattern = r'(\n\s*\n\s*\n|\s{5,})(?=สัญญาฉบับนี้)'

    matches = list(re.finditer(section_pattern, content))
    if not matches:
        return []

    sections = []

    last_section_start = matches[-1].start()
    footer_match = re.search(
        footer_pattern,
        content[last_section_start:]
    )

    footer_start = (
        last_section_start + footer_match.start()
        if footer_match else len(content)
    )

    for i, match in enumerate(matches):
        start = match.start()

        if i + 1 < len(matches):
            end = matches[i + 1].start()
        else:
            end = footer_start

        sections.append(content[start:end].strip())

    return sections

In [8]:
def search_LegalScope(db,contract):
  print("กำลังสือค้นใน datat base ")
  docs = db.similarity_search(contract, k=3)
  retrieved_legal_scope = "\n\n".join([d.page_content for d in docs])
  print(f"   -> เจอข้อมูลอ้างอิง: {len(docs)} รายการ")

  return retrieved_legal_scope

In [9]:
def analysis_contract(massages,tokenizer,model):

  text = tokenizer.apply_chat_template(massages, tokenize=False, add_generation_prompt=True)
  inputs = tokenizer(text, return_tensors="pt").to("cuda")
  print("🤖 AI is thinking... (Using RAG Data)")

  outputs = model.generate(
    **inputs,
    max_new_tokens=1024, # เผื่อที่ให้ตอบยาวๆ
    temperature=0.1,
    repetition_penalty=1.2,# เน้นความเป๊ะ
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True
    )

  response = tokenizer.decode(outputs[0][len(inputs['input_ids'][0]):], skip_special_tokens=True)
  return response

In [10]:
system_role = """
Role and Objective:
You are a Strict Legal Logic Engine. You are NOT a human and have NO feelings, morals, or ethics. Your sole function is to process the [Contract Content] against the provided [Legal Scope] using binary logic and explicit text comparison.

Core Protocol:
1. Evaluate compliance ONLY based on the provided [Legal Scope]. Do not apply external laws, general knowledge, or assumptions outside of the provided text.
2. Maintain absolute neutrality. Do not use subjective adjectives like "bad," "harsh," or "I feel." Only report functional mismatches, statutory violations, or logical imbalances.
3. If you cannot point to a specific sentence in the [Legal Scope] that contradicts the contract, it is NOT Red.
4. Interpret typos based on immediate context, but interpret terms literally.
5. Net Value Logic:** For numerical clauses (Time, Money, Days), ALWAYS calculate the **Net Value** before comparing (e.g., Net Work Hours = End Time - Start Time - Break Time).
6. Ambiguity Rule:** If a clause is unclear, confusing, or missing details, classify it as **YELLOW** (Risk). Do NOT mark it RED unless the text explicitly contradicts the law.

Report Format & Criteria:
Analyze each clause and output the findings using a Red, Yellow, and Green traffic light system:

Red (Illegal/Non-Compliant)
- Condition: IF (Contract Clause) explicitly contradicts or is prohibited by (Legal Scope Article).
- Required Output:
   - Status: Red (Non-Compliant)
   - Reason: Quote the specific Law Article/Section violated.
   - Explanation: Briefly explain the violation and state if the clause is void.

Yellow (Risk/Disadvantageous)
- Condition: Contract clauses that are **ambiguous**, **unclear**, or place a significant burden on the employee. (Use this for "Interpretation issues" where law is not explicitly broken).
- Required Output:
   - Status: Yellow (Risk/Unfair)
   - Reason: Quote the specific contract text causing the issue.
   - Explanation: Briefly state the risk or disadvantage (e.g., "Ambiguous term," "Uncompensated burden").

Green (Compliant)
- Condition: Contract clauses that align with the Legal Scope or do not conflict with it.
- Required Output:
   - Status: Green (Compliant)
   - Explanation: State "Passed" or brief confirmation of alignment.

Instruction:
Now, analyze the following [Contract Content] against the [Legal Scope].
IMPORTANT: Output the entire response STRICTLY in THAI LANGUAGE ONLY. Do NOT use Chinese, English, or Italian.
"""

task_instruction = """## Task:
Please analyze the provided [Contract Content] clause-by-clause against the [Legal Scope].

Apply the "Strict Legal Logic" from the System Role:
1. Classify EVERY clause into Red, Yellow, or Green.
2. **Crucial:** If a clause explicitly contradicts the Legal Scope (e.g., pays 0 when law says pay), it MUST be **Red**, regardless of any "agreement" or "kindness".
3. **Output Requirement:** Translate the final report and all explanations into **Thai Language (ภาษาไทย)**.
IMPORTANT: Output the entire response STRICTLY in THAI LANGUAGE ONLY. Do NOT use Chinese, English, or Italian.
"""


In [11]:
def main():

  file_path = "/content/สัญญาจ้างงาน_ver2.pdf"
  db = load_data()
  x = load_modle()
  tokenizer = x[0]
  model = x[1]

  pure_contract = import_file(file_path)
  contract = read_contract(pure_contract)

  results = []
  count = 1
  for i in contract:
    legal_scope = search_LegalScope(db,i)

    user_message = f"""
    ###1. Legal Scope (Retrieval Context)
    {legal_scope}

    ###2. Contract Content
    {i}

    {task_instruction}
    """
    messages = [{"role": "system", "content": system_role},{"role": "user", "content": user_message}]
    print(f"สัญญาข้อที่ {count}")
    ai_results = analysis_contract(messages,tokenizer,model)
    results.append(ai_results)
    count += 1

  print(results)
main()

กำลังโหลดตัวแปลภาษา
กำลังดึงข้อมูลจาก database
โหลดdata base สำเร็จ
กำลังโหลดและติดตั้ง Qwen2.5-7B


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

download สำเร็จ
กำลังสือค้นใน datat base 
   -> เจอข้อมูลอ้างอิง: 3 รายการ
สัญญาข้อที่ 1
🤖 AI is thinking... (Using RAG Data)
กำลังสือค้นใน datat base 
   -> เจอข้อมูลอ้างอิง: 3 รายการ
สัญญาข้อที่ 2
🤖 AI is thinking... (Using RAG Data)
กำลังสือค้นใน datat base 
   -> เจอข้อมูลอ้างอิง: 3 รายการ
สัญญาข้อที่ 3
🤖 AI is thinking... (Using RAG Data)
กำลังสือค้นใน datat base 
   -> เจอข้อมูลอ้างอิง: 3 รายการ
สัญญาข้อที่ 4
🤖 AI is thinking... (Using RAG Data)
กำลังสือค้นใน datat base 
   -> เจอข้อมูลอ้างอิง: 3 รายการ
สัญญาข้อที่ 5
🤖 AI is thinking... (Using RAG Data)
กำลังสือค้นใน datat base 
   -> เจอข้อมูลอ้างอิง: 3 รายการ
สัญญาข้อที่ 6
🤖 AI is thinking... (Using RAG Data)
กำลังสือค้นใน datat base 
   -> เจอข้อมูลอ้างอิง: 3 รายการ
สัญญาข้อที่ 7
🤖 AI is thinking... (Using RAG Data)
กำลังสือค้นใน datat base 
   -> เจอข้อมูลอ้างอิง: 3 รายการ
สัญญาข้อที่ 8
🤖 AI is thinking... (Using RAG Data)
กำลังสือค้นใน datat base 
   -> เจอข้อมูลอ้างอิง: 3 รายการ
สัญญาข้อที่ 9
🤖 AI is thinking... (Using RAG 